In [6]:
import pandas as pd
import py_stringmatching as sm
import math
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import style
import re
import operator
style.use('ggplot')

# read in csv as dataframe
c = pd.read_csv("../blocking/candidate_set.csv", encoding="ISO-8859-1", index_col='_id')

c.head()

,ltable_id,rtable_id,ltable_Unnamed,ltable_name,ltable_genus,ltable_family,ltable_ecology,ltable_countries,ltable_threat_paragraph,ltable_conservation_paragraph,...,scientific_name_scientific_name_lev_sim,scientific_name_scientific_name_nmw,scientific_name_scientific_name_sw,block_block_lev_dist,block_block_lev_sim,block_block_jar,block_block_jwn,block_block_exm,block_block_jac_qgm_3_qgm_3,rtable_block.1
_id,,,,,,,,,,,,,,,,,,,,,
25215,1470,3,1470,"adelaide pigmy bluetongue skink, pygmy bluetongue",Tiliqua,Scincidae,NaN,Australia;,NaN,NaN,...,1.000000,20,20,0,1,1,1,1,1,yes
75598,4363,9,4363,albanian water frog,Pelophylax,Ranidae,Terrestrial; Freshwater,Albania; Montenegro;,\n\n\n The major threat is drainage of wetland habitats and aquatic pollution of many waterwa...,"[""\n It is listed on Appendix III of the Bern Convention. 'Green frogs', including "", <em>R. ...",...,1.000000,22,22,0,1,1,1,1,1,yes
75600,4365,9,4365,NaN,Pelophylax,Ranidae,Terrestrial; Freshwater,China;,\n\n\n The major threat is over-collecting for food; other threats include drought and water ...,['\n The Shapotou location is within a national nature reserve. There is a need to manage the...,...,0.608696,13,13,0,1,1,1,1,1,yes
78778,7543,9,7543,karpathos frog,Pelophylax,Ranidae,Terrestrial; Freshwater,Greece;,\n\n\n The major threat is habitat loss and degradation of suitable aquatic habitats. Althoug...,['\n It is listed on Appendix III of the Bern Convention. It probably occurs in a protected a...,...,0.636364,13,13,0,1,1,1,1,1,yes
83513,4363,10,4363,albanian water frog,Pelophylax,Ranidae,Terrestrial; Freshwater,Albania; Montenegro;,\n\n\n The major threat is drainage of wetland habitats and aquatic pollution of many waterwa...,"[""\n It is listed on Appendix III of the Bern Convention. 'Green frogs', including "", <em>R. ...",...,1.000000,22,22,0,1,1,1,1,1,yes


In [7]:
#list col names
list(c.columns.values)

['ltable_id',
 'rtable_id',
 'ltable_Unnamed',
 'ltable_name',
 'ltable_genus',
 'ltable_family',
 'ltable_ecology',
 'ltable_countries',
 'ltable_threat_paragraph',
 'ltable_conservation_paragraph',
 'ltable_pop_trend',
 'ltable_status',
 'ltable_country_count',
 'ltable_scientific_name',
 'ltable_block',
 'rtable_Unnamed',
 'rtable_scientific_name',
 'rtable_name',
 'rtable_kingdom',
 'rtable_phylum',
 'rtable_class',
 'rtable_order',
 'rtable_family',
 'rtable_genus',
 'rtable_size',
 'rtable_threats',
 'rtable_conservation',
 'rtable_threat_keywords',
 'rtable_conservation_keywords',
 'rtable_tCount',
 'rtable_cCount',
 'rtable_block',
 '_id.1',
 'ltable_id.1',
 'rtable_id.1',
 'Unnamed_Unnamed_exm',
 'Unnamed_Unnamed_anm',
 'Unnamed_Unnamed_lev_dist',
 'Unnamed_Unnamed_lev_sim',
 'name_name_jac_qgm_3_qgm_3',
 'name_name_cos_dlm_dc0_dlm_dc0',
 'name_name_jac_dlm_dc0_dlm_dc0',
 'name_name_mel',
 'name_name_lev_dist',
 'name_name_lev_sim',
 'name_name_nmw',
 'name_name_sw',
 'family_

In [10]:
c['ltable_status'].head()

_id
25215                            \n\n\n    Endangered\n\n\n\n\n\n\n\n\n    B1+2c\n\n\n\n\n\n    
75598                        \n\n\n    Endangered\n\n\n\n\n\n\n\n\n    B1ab(iii)\n\n\n\n\n\n    
75600                      \n\n\n    Endangered\n\n\n\n\n\n\n\n\n    B1ab(iii,v)\n\n\n\n\n\n    
78778    \n\n\n    Critically Endangered\n\n\n\n\n\n\n\n\n    B1ab(iii)+2ab(iii)\n\n\n\n\n\n    
83513                        \n\n\n    Endangered\n\n\n\n\n\n\n\n\n    B1ab(iii)\n\n\n\n\n\n    
Name: ltable_status, dtype: object

In [11]:
# clean up ltable_status column
# author: @andrewedstrom

# remove parenthesis contents
for index, row in c.iterrows():
    status = row['ltable_status']
    if 'Critically' in status:
        status = 'Critically Endangered'
    else:
        status = 'Endangered'
    c.loc[index, 'ltable_status'] = status
    
c.head()

,ltable_id,rtable_id,ltable_Unnamed,ltable_name,ltable_genus,ltable_family,ltable_ecology,ltable_countries,ltable_threat_paragraph,ltable_conservation_paragraph,...,scientific_name_scientific_name_lev_sim,scientific_name_scientific_name_nmw,scientific_name_scientific_name_sw,block_block_lev_dist,block_block_lev_sim,block_block_jar,block_block_jwn,block_block_exm,block_block_jac_qgm_3_qgm_3,rtable_block.1
_id,,,,,,,,,,,,,,,,,,,,,
25215,1470,3,1470,"adelaide pigmy bluetongue skink, pygmy bluetongue",Tiliqua,Scincidae,NaN,Australia;,NaN,NaN,...,1.000000,20,20,0,1,1,1,1,1,yes
75598,4363,9,4363,albanian water frog,Pelophylax,Ranidae,Terrestrial; Freshwater,Albania; Montenegro;,\n\n\n The major threat is drainage of wetland habitats and aquatic pollution of many waterwa...,"[""\n It is listed on Appendix III of the Bern Convention. 'Green frogs', including "", <em>R. ...",...,1.000000,22,22,0,1,1,1,1,1,yes
75600,4365,9,4365,NaN,Pelophylax,Ranidae,Terrestrial; Freshwater,China;,\n\n\n The major threat is over-collecting for food; other threats include drought and water ...,['\n The Shapotou location is within a national nature reserve. There is a need to manage the...,...,0.608696,13,13,0,1,1,1,1,1,yes
78778,7543,9,7543,karpathos frog,Pelophylax,Ranidae,Terrestrial; Freshwater,Greece;,\n\n\n The major threat is habitat loss and degradation of suitable aquatic habitats. Althoug...,['\n It is listed on Appendix III of the Bern Convention. It probably occurs in a protected a...,...,0.636364,13,13,0,1,1,1,1,1,yes
83513,4363,10,4363,albanian water frog,Pelophylax,Ranidae,Terrestrial; Freshwater,Albania; Montenegro;,\n\n\n The major threat is drainage of wetland habitats and aquatic pollution of many waterwa...,"[""\n It is listed on Appendix III of the Bern Convention. 'Green frogs', including "", <em>R. ...",...,1.000000,22,22,0,1,1,1,1,1,yes


In [13]:
c['rtable_country']

KeyError: 'rtable_country'

In [ ]:
# add Status match column
# author: @andrewedstrom
import operator, re

statusColumn = []
for index, row in c.iterrows():
    lstatus = row['ltable_status']
    rstatus = row['rtable_status']
    
    if lstatus is rstatus:
        statusColumn.append(1)
    else:
        statusColumn.append(0)

df['status_match'] = statusColumn
df.head()

In [ ]:
# add country match column
# author: @andrewedstrom
import operator, re, string
country_match_column = []

#create delimiter tokenizer using semicolon as delimiter
delim_tok = sm.DelimiterTokenizer(delim_set=['; '], return_set=True)

#create overlap coefficient similarity measure
oc = sm.OverlapCoefficient()

def clean_tokens(tok_list):
    new_list = []
    for s in tok_list:
        s = s.lower()
        
        #parentheticals
        if '(' in s:
            pstart = s.find('(')
            pend = s.find(')')
            if pend < len(s) - 1 and pend > 0:
                s = s[0:pstart] + s[pend+1:]
            else:
                s = s[0:pstart]
                
        #leading/trailing whitespace
        s = s.strip()
        
        #leading/trailing punctuation
        s = s.strip(string.punctuation)
        new_list.append(s)
    return new_list

for index, row in c.iterrows():
    lc = row['ltable_countries']
    rc = row['rtable_countries']
    
    ltok = delim_tok.tokenize(lc)
    rtok = delim_tok.tokenize(rc)
    ltok = clean_tokens(ltok)
    rtok = clean_tokens(rtok)
    
    overlap = oc.get_raw_score(ltok, rtok)
    
    country_match_column.append(overlap)
    
    
c['country_overlap'] = country_match_column
c.head()

In [ ]:
#c.to_csv('../blocking/candidate_set.csv')